In [2]:
#import required packages

In [3]:

#basics
import pandas as pd 
import numpy as np

#misc
import gc
import time
import warnings

#stats
#from scipy.misc import imread
from scipy import sparse
import scipy.stats as ss

#viz
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import seaborn as sns                        #pip install wordcloud
from PIL import Image
#import matplotlib_venn as venn

#nlp
import nltk
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords

#import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   

from imblearn.over_sampling import RandomOverSampler

#FeatureEngineering
#!pip install lightgbm
#from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas.util.testing as tm

import  textblob                   #pip install textblob
#import xgboost
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

from textblob import TextBlob
from nltk.stem import PorterStemmer
import nltk
    #nltk.download('wordnet')
from textblob import Word

#settings
start_time=time.time()
color = sns.color_palette()
sns.set_style("dark")
eng_stopwords = set(stopwords.words("english"))
warnings.filterwarnings("ignore")

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

%matplotlib inline

C:\Users\saura\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Importing Dataset

In [5]:
train=pd.read_csv("F:/Toxic_Classification/train.csv")
train_text=train.comment_text

In [6]:
test=pd.read_csv("F:/Toxic_Classification/test.csv",encoding='latin9')

In [7]:
test_label=pd.read_csv("F:/Toxic_Classification/test_labels.csv",encoding='latin9')

In [8]:
test=pd.merge(test,test_label,on='id',how='left')


In [9]:
# Data preprocessing(cleaning)

In [10]:
test.isna().sum()

id                0
comment_text      0
toxic            48
severe_toxic     48
obscene          48
threat           48
insult           48
identity_hate    48
dtype: int64

In [11]:
test_pred=test[test['toxic'].isna()]   ## 48 comments for which o/p not given i.e. need to be predicted

In [12]:
test=test.dropna(axis=0).reset_index(drop=True)

In [13]:
test_text=test.comment_text

In [14]:
Total_text=pd.concat([train_text,test_text],axis=0).reset_index(drop=True)

In [15]:
Total_text

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
223496    :Jerome, I see you never got around to thisâŠ...
223497    ==Lucky bastard== \n http://wikimediafoundatio...
223498    ==shame on you all!!!== \n\n You want to speak...
223499    MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...
223500    " \n\n == Unicorn lair discovery == \n\n Suppo...
Name: comment_text, Length: 223501, dtype: object

In [16]:
#Abbrevations and Words correction
def clean_text(text):
    text = text.lower()
    text = text.strip()
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"[-()\"#/@;:{}`+=~|.!?,'0-9]", "", text)
    return(text)

In [17]:
stop = set(nltk.corpus.stopwords.words('english'))

nltk.corpus.stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [18]:
import string
def pre_process(text):
    text = text.str.replace('/','')                           #Replacing the / with none
    text = text.apply(lambda x: re.sub("  "," ", x))          #Replacing double space with single space
    #text = re.sub(r"[-()\"#/@;:{}`+=~|.!?,']", "", text)      #Replacing special character with none
    #text = re.sub(r'[0-9]+', '', text)                        #Replacing numbers with none
    #text = text.apply(lambda x: " ".join(x.translate(str.maketrans('', '', string.punctuation)) for x in x.split() if x.isalpha()))
    text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #Removing stop words
    #text = text.apply(lambda x: str(TextBlob(x).correct()))                      #Correct spelling corrections
    #text = text.apply(lambda x: " ".join(PorterStemmer().stem(word) for word in x.split())) #Stemming using porter stemmer
    #text = text.apply(lambda x: " ".join(stemmer_func(word) for word in x.split()))        #Stemming
    #text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))   #lemmatization
    #text = text.apply(lambda x: " ".join(word for word, pos in pos_tag(x.split()) if pos not in ['NN','NNS','NNP','NNPS'])) #Removing nouns etc
    return(text)

In [19]:
Total_text = Total_text.apply(lambda x: clean_text(x))


In [20]:
Total_text=pre_process(Total_text)


In [21]:
# Converting text to numbers(vectorising) using ngram tf-idf(term frequency-inverse document frequency)

In [22]:
# ngram  tf-idf 
tfidf_vect_ngram = CountVectorizer(analyzer='word', token_pattern='\w{1,}', ngram_range=(1, 2),max_features=800)
tfidf_vect_ngram.fit(Total_text)


CountVectorizer(max_features=800, ngram_range=(1, 2), token_pattern='\\w{1,}')

In [23]:
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_text)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(test_text)

In [24]:
# user defined function for specifying the type of classifier and input & output variables for training the model and prediction 

In [25]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,  valid_y):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return metrics.accuracy_score(classifier.predict(feature_vector_train), label), metrics.accuracy_score(predictions, valid_y)

In [26]:
test.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [27]:
## preprocessing the test dataset(48 nos.) and converting text to numbers

In [28]:
test_pred=test_pred.drop(columns='id').reset_index(drop=True)

In [29]:
test_pred

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,And you as well. I also found it interesting t...,NaN,NaN,NaN,NaN,NaN,NaN
1,":::Khurram, I have removed your personal attac...",NaN,NaN,NaN,NaN,NaN,NaN
2,""" \n\n == Begging your pardon for all this... ...",NaN,NaN,NaN,NaN,NaN,NaN
3,""" May 2014 (UTC) \n *Comment a lot of the oppo...",NaN,NaN,NaN,NaN,NaN,NaN
4,""" \n\n \n == """"hate speech"""" == \n\n Can some...",NaN,NaN,NaN,NaN,NaN,NaN
5,== GAY FCK ME == \n\n You are not currently lo...,NaN,NaN,NaN,NaN,NaN,NaN
6,"â Basically, what Kizzle said. \n :1 It's ...",NaN,NaN,NaN,NaN,NaN,NaN
7,"""==March 2010== \n | """,NaN,NaN,NaN,NaN,NaN,NaN
8,::::OK. I've packed a bunch of exact reference...,NaN,NaN,NaN,NaN,NaN,NaN
9,""":Xingdong said """"wikipedia is a good place to...",NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
test_pred_txt = test_pred.comment_text.apply(lambda x: clean_text(x))
test_pred_txt=pre_process(test_pred_txt)

In [31]:
xtest_pred_tfidf_ngram =  tfidf_vect_ngram.transform(test_pred_txt)

In [32]:
# model training,accuracy check and predicting for the multiple outputs for test dataset(48 nos.) using for loop 

In [33]:
for iv in train.iloc[:,2:].columns:
    model=LogisticRegression()
    model.fit(xtrain_tfidf_ngram.toarray(),train[iv])
    predictions=model.predict(xtest_tfidf_ngram.toarray())
    test_pred[iv]=model.predict(xtest_pred_tfidf_ngram.toarray())
    print('Accuracy for train & test respectively for '+iv,metrics.accuracy_score(model.predict(xtrain_tfidf_ngram.toarray()) ,train[iv]), metrics.accuracy_score(predictions, test[iv]))

Accuracy for train & test respectively for toxic 0.944745599137688 0.9251837947755357
Accuracy for train & test respectively for severe_toxic 0.9906624637308784 0.9940872829657438
Accuracy for train & test respectively for obscene 0.9738486316435944 0.9567339277334584
Accuracy for train & test respectively for threat 0.9969606006103866 0.9963866729235101
Accuracy for train & test respectively for insult 0.9611395554330048 0.9523541373377131
Accuracy for train & test respectively for identity_hate 0.9911074067343063 0.9882214922571563


In [34]:
## prediction for test dataset(48nos.)

In [35]:
test_pred

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,And you as well. I also found it interesting t...,0,0,0,0,0,0
1,":::Khurram, I have removed your personal attac...",0,0,0,0,0,0
2,""" \n\n == Begging your pardon for all this... ...",0,0,0,0,0,0
3,""" May 2014 (UTC) \n *Comment a lot of the oppo...",0,0,0,0,0,0
4,""" \n\n \n == """"hate speech"""" == \n\n Can some...",1,0,0,0,0,0
5,== GAY FCK ME == \n\n You are not currently lo...,0,0,0,0,0,0
6,"â Basically, what Kizzle said. \n :1 It's ...",0,0,0,0,0,0
7,"""==March 2010== \n | """,0,0,0,0,0,0
8,::::OK. I've packed a bunch of exact reference...,0,0,0,0,0,0
9,""":Xingdong said """"wikipedia is a good place to...",0,0,0,0,0,0
